## Image Caption Generation v2

In [1]:
from pycocotools.coco import COCO
import pandas as pd
import numpy as np
import cv2
from picturedata.generator import DataGenerator
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Merge, Activation, Flatten, GRU
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras import backend as K

Using TensorFlow backend.


In [2]:
annFile_train = "annotations/captions_train2014.json"
annFile_val = "annotations/captions_val2014.json"

coco_train = COCO(annFile_train)
coco_val = COCO(annFile_val)

loading annotations into memory...
Done (t=1.72s)
creating index...
index created!
loading annotations into memory...
Done (t=1.09s)
creating index...
index created!


In [3]:
imgIds_train = coco_train.getImgIds()
imgIds_val = coco_val.getImgIds()

annIds_train = coco_train.getAnnIds(imgIds_train)
annIds_val = coco_val.getAnnIds(imgIds_val)
anns_train = coco_train.loadAnns(annIds_train)
anns_val = coco_val.loadAnns(annIds_val)

imgs_train = coco_train.loadImgs(imgIds_train)
imgs_val = coco_val.loadImgs(imgIds_val)

In [4]:
imgs_train_df = pd.DataFrame(imgs_train)
imgs_val_df = pd.DataFrame(imgs_val)

anns_train_df = pd.DataFrame(anns_train)
anns_val_df = pd.DataFrame(anns_val)

In [5]:
img_train_new = imgs_train_df.drop(['coco_url','date_captured','flickr_url','height','width','license'], axis=1)
img_val_new = imgs_val_df.drop(['coco_url','date_captured','flickr_url','height','width','license'], axis=1)

img_train_new.rename(columns={'id': 'image_id'}, inplace=True)
img_val_new.rename(columns={'id': 'image_id'}, inplace=True)

anns_train_df['caption'] = anns_train_df['caption'].apply(lambda x: '<start>' + x + '<end>')
anns_val_df['caption'] = anns_val_df['caption'].apply(lambda x: '<start>' + x + '<end>')

train_df = pd.merge(anns_train_df, img_train_new, on='image_id', how='left')
val_df = pd.merge(anns_val_df, img_val_new, on='image_id', how='left')

In [7]:
train_df.drop(['id'], axis=1, inplace=True)
val_df.drop(['id'], axis=1, inplace=True)

In [8]:
train_imgs = set(train_df['file_name'])
len(train_imgs)*5

413915

In [9]:
img_dir = "images/"
images = []
for img_file in train_imgs:
    image = cv2.imread(img_dir + img_file)
    image.resize((3,224,224))
    for i in range(0,5):
        images.append(image)


In [9]:
captions = train_df['caption']

In [10]:
generator = DataGenerator(images, captions)

Total samples : 3975778
Vocabulary size: 54028
Maximum caption length: 50


In [11]:
embedding_dim = 128

In [29]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
base_model.trainable=False
image_model = Sequential()
image_model.add(base_model)
image_model.add(Flatten())
image_model.add(Dense(embedding_dim, input_dim=4096, activation='relu'))

image_model.add(RepeatVector(generator.getMaxCapLen()))

language_model = Sequential()
language_model.add(Embedding(generator.getVocabSize(), 256, input_length=generator.getMaxCapLen()))
language_model.add(LSTM(256, return_sequences=True))
#language_model.add(GRU(128, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_dim)))

model = Sequential()
model.add(Merge([image_model, language_model], mode='concat'))
model.add(LSTM(1000, return_sequences=False))
#model.add(GRU(256, return_sequences=False))
model.add(Dense(generator.getVocabSize()))
model.add(Activation('softmax'))


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:17: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [30]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [34]:
file_name = 'weights-improvement-{epoch:02d}.hdf5'
checkpoint = ModelCheckpoint(file_name, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit_generator(generator.generate(batch_size=32), steps_per_epoch=50, epochs=50, verbose=1, callbacks=callbacks_list)
try:
    model.save('Models/WholeModel.h5', overwrite=True)
    model.save_weights('Models/Weights.h5',overwrite=True)
except:
    print "Error in saving model."

Generating data...Epoch 1/50

50/50 [==============================] - 94s - loss: 6.0786 - acc: 0.0931    
Epoch 2/50
50/50 [==============================] - 94s - loss: 7.6872 - acc: 0.0950    
Epoch 3/50
50/50 [==============================] - 95s - loss: 7.7185 - acc: 0.1081    
Epoch 4/50
50/50 [==============================] - 95s - loss: 7.8049 - acc: 0.1000    
Epoch 5/50
50/50 [==============================] - 95s - loss: 6.9789 - acc: 0.1013    
Epoch 6/50
50/50 [==============================] - 95s - loss: 7.3835 - acc: 0.0956    
Epoch 7/50
50/50 [==============================] - 95s - loss: 7.3623 - acc: 0.0875    
Epoch 8/50
50/50 [==============================] - 95s - loss: 7.7563 - acc: 0.0862    
Epoch 9/50
50/50 [==============================] - 95s - loss: 7.9194 - acc: 0.0950    
Epoch 10/50
50/50 [==============================] - 95s - loss: 7.6188 - acc: 0.1163    
Epoch 11/50
50/50 [==============================] - 95s - loss: 7.5596 - acc: 0.0975    
